In [ ]:
!pip install -U skorch
!pip install d2l
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
!pip install pycountry_convert

import torch
from torch import nn
import pandas as pd
import numpy as np
from d2l import torch as d2l
import json
from google.colab import files
import pycountry_convert as pc
from IPython import display


api_token = {"username":"k0ndr4","key":"9ec1191895c870bb0505c7918e027ba7"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c tp-n2-aprendizaje-profundo-2021-by-datitos-v2
!unzip fifa2021_test.csv.zip
!unzip fifa2021_training.csv.zip

# Cargamos archivos 
train_data = pd.read_csv('fifa2021_training.csv')
test_data = pd.read_csv('fifa2021_test.csv')
print(train_data.shape)
print(test_data.shape)

     |████████████████████████████████| 133kB 5.6MB/s 
     |████████████████████████████████| 81kB 3.1MB/s 
     |████████████████████████████████| 10.1MB 5.7MB/s 
     |████████████████████████████████| 245kB 47.1MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=b97a75302969c9af99e99be3a104ff24798d08b12a1b9c54f75c9139983b061b
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built pycountry
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3.6, but you'll have coverage 5.5 which is incompatible.
ERROR: pytest-cov 2.11.1 has requirement pytest>=4.6, but you'll have pytest 3.6.4 which is incompatible.
ERROR: pytest-mock 3.5.1 has requirement p

In [ ]:
def country_to_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except:
        d = {'Korea Republic': 'Asia',
             'Wales': 'Europe',
             'Bosnia & Herzegovina': 'Europe'}
        pass#return d[country_name]

# Example
country_name = 'England'
print(country_to_continent(country_name))

#Concatenamos data sets y eliminamos columnas que no aportan información
all_features = pd.concat((train_data.iloc[:, :-1], test_data))
all_features['Continent'] = [country_to_continent(i) for i in all_features['Natinality'].values]

features_to_drop = ['ID',
                    'Name',
                    'Club',
                    'BirthDate',
                    'Club_KitNumber',
                    'Club_JoinedClub',
                    'Club_ContractLength',
                    'Natinality'
                    ]

all_features = all_features.drop(features_to_drop, axis=1)
all_features.head()

#Visualizamos tipos de datos de nuestro data frame
#all_features.dtypes

#Contabilizamos valores nulos
#all_features.isna().sum()

None


,Overal,Potential,Height,Weight,PreferredFoot,Age,PlayerWorkRate,WeakFoot,SkillMoves,Value,Wage,BallControl,Dribbling,Marking,SlideTackle,StandTackle,Aggression,Reactions,Interceptions,Vision,Composure,Crossing,ShortPass,LongPass,Acceleration,Stamina,Strength,Balance,SprintSpeed,Agility,Jumping,Heading,ShotPower,Finishing,LongShots,Curve,FKAcc,Penalties,Volleys,GKDiving,GKHandling,GKKicking,GKReflexes,Sex,Continent
0,64,73,188,79,R,22,Medium/Medium,3,2,1200000.0,1500.0,62,55,60,57,60,71,58,60,61,63,44,67,64,54,66,70,54,46,58,57,53,55,39,43,39,33,43,41,13,10,13,12,Male,Europe
1,67,70,172,64,R,25,High/Medium,3,3,1400000.0,3300.0,66,68,33,34,33,37,64,20,58,67,64,62,57,84,65,57,78,80,80,42,49,75,60,69,64,61,65,59,12,9,6,15,Male,Europe
2,68,68,173,70,L,28,Medium/Medium,2,2,1100000.0,3400.0,62,63,62,63,64,74,64,72,54,61,62,63,56,74,75,64,72,68,66,81,62,68,39,62,58,34,49,46,15,6,15,16,Male,North America
3,63,70,190,76,L,22,Medium/High,3,2,900000.0,1800.0,36,31,69,68,72,55,55,57,36,48,35,49,48,62,68,70,66,61,53,71,59,43,28,29,25,29,48,39,9,12,11,6,Male,South America
4,69,72,178,68,R,25,High/Medium,3,3,1800000.0,6400.0,69,73,47,43,36,46,59,27,64,53,65,62,65,79,74,44,73,79,78,55,45,57,62,44,56,43,56,53,8,11,10,15,Male,South America


In [ ]:
#Estandarizamos valores numéricos
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features].isna().sum()

all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(all_features[numeric_features].mean())

# `Dummy_na=True` considers "na" (missing value) as a valid feature value, and creates an indicator feature for it
all_features = pd.get_dummies(all_features, dummy_na=True)

#all_features.shape
all_features.head()

,Overal,Potential,Height,Weight,Age,WeakFoot,SkillMoves,Value,Wage,BallControl,Dribbling,Marking,SlideTackle,StandTackle,Aggression,Reactions,Interceptions,Vision,Composure,Crossing,ShortPass,LongPass,Acceleration,Stamina,Strength,Balance,SprintSpeed,Agility,Jumping,Heading,ShotPower,Finishing,LongShots,Curve,FKAcc,Penalties,Volleys,GKDiving,GKHandling,GKKicking,GKReflexes,PreferredFoot_L,PreferredFoot_R,PreferredFoot_nan,PlayerWorkRate_High/High,PlayerWorkRate_High/Low,PlayerWorkRate_High/Medium,PlayerWorkRate_Low/High,PlayerWorkRate_Low/Low,PlayerWorkRate_Low/Medium,PlayerWorkRate_Medium/High,PlayerWorkRate_Medium/Low,PlayerWorkRate_Medium/Medium,PlayerWorkRate_nan,Sex_Female,Sex_Male,Sex_nan,Continent_Africa,Continent_Asia,Continent_Europe,Continent_North America,Continent_Oceania,Continent_South America,Continent_nan
0,-0.336304,0.223299,0.981364,0.567831,-0.776243,0.053378,-0.498472,-0.259597,-0.404799,0.161472,-0.071844,0.624642,0.510397,0.537972,0.857286,-0.466648,0.612180,0.447157,0.316933,-0.348041,0.500534,0.668093,-0.724207,0.177275,0.399455,-0.730841,-1.294080,-0.391126,-0.663421,0.029989,-0.273676,-0.380337,-0.233531,-0.489547,-0.574249,-0.350375,-0.129824,-0.199792,-0.369944,-0.191595,-0.259047,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0
1,0.066724,-0.239964,-1.286506,-1.483312,-0.144030,0.053378,0.764264,-0.239724,-0.331185,0.397767,0.609690,-0.690010,-0.574973,-0.708979,-1.109803,0.158119,-1.286996,0.233509,0.637426,0.735812,0.162727,0.212172,1.285527,0.113895,-0.643419,0.977384,1.030186,1.117175,-1.945380,-0.196729,1.221959,0.669891,1.087673,0.854342,1.016228,1.023563,0.863712,-0.255665,-0.428248,-0.607516,-0.094262,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0.201067,-0.548806,-1.144764,-0.662855,0.488182,-1.420827,-0.498472,-0.269533,-0.327095,0.161472,0.347562,0.722023,0.793537,0.722705,1.030853,0.158119,1.181934,-0.051356,0.156687,0.627427,0.230288,0.147040,0.615616,0.747693,-0.081872,0.550328,0.209857,0.157347,1.387714,0.540105,0.698487,-0.380337,0.731964,0.531809,-0.517446,0.024335,0.146158,-0.088047,-0.603160,-0.072760,-0.039333,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0
3,-0.470647,-0.239964,1.264848,0.157602,-0.776243,0.053378,-0.498472,-0.289405,-0.392530,-1.374449,-1.330061,1.062859,1.029487,1.092172,-0.068403,-0.779031,0.469742,-1.333247,-0.884915,-0.835775,-0.715572,-0.374012,-0.188278,0.304034,0.399455,0.123271,-0.268668,-0.733922,0.533074,0.370066,-1.171057,-0.930457,-0.944949,-1.242125,-0.801460,-0.038117,-0.240217,-0.423284,-0.253336,-0.310429,-0.588617,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0
4,0.335410,0.068878,-0.436055,-0.936340,-0.144030,0.053378,0.764264,-0.199979,-0.204405,0.574989,0.871819,-0.008339,-0.150263,-0.570429,-0.589103,-0.362520,-0.954640,0.660806,-0.484299,0.790005,0.162727,0.733224,0.950571,0.684313,-1.686294,0.621504,0.961825,0.980057,-0.834349,-0.423447,-0.124112,0.769912,-0.182716,0.424298,-0.006222,0.461497,0.532533,-0.479156,-0.311640,-0.369847,-0.094262,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


In [ ]:
print(train_data['Position'].value_counts())
# Convierto la predicción a un número
train_data.Position = train_data.Position.astype('category')
print(train_data[['Position']])
dtype = pd.CategoricalDtype(['DEF', 'FWD', 'GK', 'MID'], ordered=False)
train_data['Position'] = train_data.Position.cat.codes
#print(train_data[['Position']])

MID    5071
DEF    4585
FWD    2715
GK     1550
Name: Position, dtype: int64
      Position
0          MID
1          MID
2          DEF
3          DEF
4          MID
...        ...
13916      MID
13917      MID
13918      MID
13919      DEF
13920      FWD

[13921 rows x 1 columns]


In [ ]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
train_labels = torch.tensor(train_data.Position.values, dtype=torch.long)#.reshape(-1, 1), dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split (train_features, train_labels, train_size=0.95)
Y_train

tensor([3, 3, 3,  ..., 3, 3, 0])

In [ ]:
in_features = train_features.shape[1]
out_features = 4 #'DEF', 'FWD', 'GK', 'MID'


In [ ]:
class MyModule(nn.Module):
    def __init__(self, dropout = 0.5,nonlin=nn.ReLU()):
        
        super(MyModule, self).__init__()
        
        self.dense0 = nn.Linear(in_features, 1024)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(1024, 2048)
        self.dense2 = nn.Linear(2048, 512)
        self.dense3 = nn.Linear(512, 128)
        self.output = nn.Linear(128, out_features)
        

    def forward(self, X, **kwargs):
        
        
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense2(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense3(X))
        X = self.dropout(X)
        X = self.output(X)
        return X

In [ ]:
from torch import optim
params = {
	'max_epochs':[8,10,12,14],
	'lr': [1e-3, 1e-4, 1e-5, 1e-6],
	'module__dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
	'optimizer': [optim.Adam, optim.RMSprop],
  #'batch_size': [1000, 2000, 3000, 4000, 4096, 5000, 6000]
  }

In [ ]:
import skorch
from skorch import NeuralNetClassifier
net = NeuralNetClassifier(
    MyModule,
    max_epochs=20,
    lr=0.001,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    optimizer__weight_decay= 1e-6,
    batch_size = 4096
)

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(net, params, refit = False, scoring= 'balanced_accuracy' , cv = 10)

In [ ]:
gs.fit(X_train, Y_train)
print(gs.best_score_, gs.best_params_)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.0059           nan  4.2793
      2           nan       0.1121           nan  3.7332
      3           nan       0.1121           nan  3.7596
      4           nan       0.1121           nan  3.8455
      5           nan       0.1121           nan  3.7352
      6           nan       0.1121           nan  3.7804
      7           nan       0.1121           nan  3.7857
      8           nan       0.1121           nan  3.8240
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.6627           nan  3.8096
      2           nan       0.3641           nan  3.7509
      3           nan       0.3641           nan  3.8312
      4           nan       0.3641           nan  3.7511
      5           nan       0.3641           nan  3.8149
      6           nan       0.3